<a href="https://colab.research.google.com/github/AryanWadhwa05/Stock-Analysis-and-Prediction/blob/main/Sentimental%20Analysis%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment
!pip install langdetect
!pip install emoji
!pip install praw
import praw

     |████████████████████████████████| 125 kB 4.9 MB/s 
     |████████████████████████████████| 981 kB 5.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=46c6350f2b5c2a2305f1e2a7bffd311deaf92fd5d7817dfae23e45d57db7b933
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
     |████████████████████████████████| 174 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=c6e36d12ef92d5bcf31546e5c8f07e76fc3cdfc0c619779ab30b8e2ab952ac91
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji
     |████████████████████████████████| 176 kB 5.0 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 


In [2]:
reddit_id = "wpotSHZd6nAbXpA-aqVTwQ"
reddit_secret = "clfa5nAXLOr3iHpbwm9RGrG7eFJZvg"
reddit_user = "HolyShoter"
reddit_pass = "GARgar109$"

In [3]:
reddit = praw.Reddit(
    client_id=reddit_id,
    client_secret=reddit_secret,
    password=reddit_pass,
    user_agent="USERAGENT",
    username=reddit_user,
    check_for_async = False
)

In [4]:
new_words = {
    'citron': -4.0,  
    'hindenburg': -4.0,        
    'moon': 4.0,
    'highs': 2.0,
    'mooning': 4.0,
    'long': 2.0,
    'short': -2.0,
    'call': 4.0,
    'calls': 4.0,    
    'put': -4.0,
    'puts': -4.0,    
    'break': 2.0,
    'tendie': 2.0,
     'tendies': 2.0,
     'town': 2.0,     
     'overvalued': -3.0,
     'undervalued': 3.0,
     'buy': 4.0,
     'sell': -4.0,
     'gone': -1.0,
     'gtfo': -1.7,
     'paper': -1.7,
     'bullish': 3.7,
     'bearish': -3.7,
     'bagholder': -1.7,
     'stonk': 1.9,
     'green': 1.9,
     'money': 1.2,
     'print': 2.2,
     'rocket': 2.2,
     'bull': 2.9,
     'bear': -2.9,
     'pumping': -1.0,
     'sus': -3.0,
     'offering': -2.3,
     'rip': -4.0,
     'downgrade': -3.0,
     'upgrade': 3.0,     
     'maintain': 1.0,          
     'pump': 1.9,
     'hot': 1.5,
     'drop': -2.5,
     'rebound': 1.5,  
     'crack': 2.5,
     "up":2.7,
     "down":-2.7,
     "comeback":3,
     "fall":-3,
     "asshole":-4,
     "suckers":-4,
     "fault":-3,
     "doesn't care":-4,
     }

In [7]:
import requests
import os
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langdetect import detect
import re
import emoji
import numpy as np
import praw
import statistics as stats
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

analyzer = SentimentIntensityAnalyzer()
analyzer.lexicon.update(new_words)

bearer_token = "AAAAAAAAAAAAAAAAAAAAAGl9WAEAAAAAup10asDXc%2BPNXX7yTpSv%2B%2B0nJQ0%3DU2xJJZhYx8LHXFfEm4NqzTNHtrCRjRufnTjw9fZcdggkQGDl0O"

def replace_RT(tweet, default_replace=""):
  tweet = re.sub('RT\s+', default_replace, tweet)
  return tweet

def replace_user(tweet, default_replace=""):
  tweet = re.sub('\B@\w+', default_replace, tweet)
  return tweet

def replace_url(tweet, default_replace=""):
  tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)
  return tweet

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "TSLA", "tag": "TSLA"},
        {"value": "Elon", "tag": "Elon"},
        {"value": "Musk", "tag": "Musk"},
        {"value": "Tesla", "tag": "Tesla"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    lst = []
    KEYWORDS = ["tesla","tsla","elon","musk"]
    def reddit_stream(KEYWORDS = KEYWORDS,amount_of_reddit = 50):
      for i in range(len(KEYWORDS)):
          try:
            for comment in reddit.subreddit(KEYWORDS[i]).stream.comments():
              sentence = comment.body
              display_sentence = comment.body
              sentence = replace_RT(sentence) # replace retweet
              sentence = replace_user(sentence) # replace user tag
              sentence = replace_url(sentence) # replace url
              sentence = sentence.lower()
              sentiment_reddit = analyzer.polarity_scores(sentence)
              if len(lst) < amount_of_reddit*(i+1):
                print("NEW " + KEYWORDS[i] + ": " + display_sentence + " " + str(sentiment_reddit["compound"]))
                lst.append(sentiment_reddit["compound"])
              else:
                break
          except:
            continue
      print(len(lst))
    reddit_stream()
# and len(lst) <= len(KEYWORDS)*40+200
    lst_twitter = []
    def twitter_streams(amount_tweets = 100):
      for response_line in response.iter_lines():
          if response_line:
            json_response = json.loads(response_line)
            tweet = json_response["data"]["text"]
            tweet = replace_RT(tweet) # replace retweet
            tweet = replace_user(tweet) # replace user tag
            tweet = replace_url(tweet) # replace url
            tweet = tweet.lower()
            sentiment = analyzer.polarity_scores(tweet)
      
            try:
              if detect(tweet) == "en" and len(lst_twitter) <= amount_tweets:
                print("New Twitter: " + tweet + " " + str(sentiment["compound"]))
                lst_twitter.append(sentiment["compound"])
                print(len(lst_twitter))
              elif len(lst_twitter) > amount_tweets:
                break
              else:
                pass
            except:
              pass
    twitter_streams()
      
    def web_scraping_sentiment():
      finviz_url = "https://finviz.com/quote.ashx?t="
      apewisdom_url = "https://apewisdom.io/stocks/"
      ticker = "TSLA"
      finviz_url = finviz_url + ticker
      apewisdom_url = apewisdom_url + ticker + "/"

      finviz_req = Request(url=finviz_url, headers={'user-agent': 'my-app/0.0.1'})
      finviz_response = urlopen(finviz_req)
      apewisdom_req = Request(url=apewisdom_url, headers = {"user-agent": "my-app/0.0.1"})
      apewisdom_response = urlopen(apewisdom_req)
      df = pd.DataFrame(columns=['News_Title', 'Time'])
      sentiment_list = []
      news_table = {}


      soup = BeautifulSoup(apewisdom_response)
      html = BeautifulSoup(finviz_response)


      mentioning_users = soup.findAll("div",{"class":"details-small-tile"})[-2]
      upvotes = soup.findAll("div",{"class":"details-small-tile"})[-3]
      mentions = soup.findAll("div",{"class":"details-small-tile"})[-4]
      news_table = html.find(id='news-table')

      mentioning_users_percentage = mentioning_users.find("span").text
      upvotes_percentage = upvotes.find("span").text
      mentions_percentage = mentions.find("span").text
      sentiment = soup.findAll("div",{"class":"tile-value"})[-1].text


      dataRows = news_table.findAll('tr')


      for i, table_row in enumerate(dataRows):
          a_text = table_row.a.text
          td_text = table_row.td.text
          
          df = df.append({'News_Title': a_text, 'Time': td_text}, ignore_index=True)


      for i in range(50):
        word = df["News_Title"][i]
        news_title_sentiment = analyzer.polarity_scores(word)
        news_title_sentiment = news_title_sentiment["compound"]
        sentiment_list.append(news_title_sentiment)

      sentiment_list = stats.mean(sentiment_list)


      sentiment = sentiment[0:2]
      sentiment = int(sentiment) - 50
      if sentiment > 1:
        sentiment = sentiment*2/100
      elif sentiment < -1:
        sentiment = sentiment*2/100

      global final_sentiment
      final_sentiment = sentiment + sentiment_list
      print(final_sentiment)
    web_scraping_sentiment()



    def find_mean_from_list():
      global final_list
      final_list = []
      final_list = lst + lst_twitter
      print(len(final_list))
      final_list = stats.mean(final_list)
      print("The mean is: " + str(final_list))
    find_mean_from_list()

    very_last_sentiment = (final_list + final_sentiment)/2
    print("\n" + str(very_last_sentiment))

def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)


if __name__ == "__main__":
    main()

{"data": [{"id": "1484015500707381248", "value": "Tesla", "tag": "Tesla"}, {"id": "1484015500707381249", "value": "TSLA", "tag": "TSLA"}, {"id": "1484015500707381250", "value": "Elon", "tag": "Elon"}, {"id": "1484015500707381251", "value": "Musk", "tag": "Musk"}], "meta": {"sent": "2022-01-20T04:11:08.394Z", "result_count": 4}}
{"meta": {"sent": "2022-01-20T04:11:09.504Z", "summary": {"deleted": 4, "not_deleted": 0}}}
{"data": [{"value": "Tesla", "tag": "Tesla", "id": "1484015617657257987"}, {"value": "TSLA", "tag": "TSLA", "id": "1484015617657257986"}, {"value": "Musk", "tag": "Musk", "id": "1484015617657257984"}, {"value": "Elon", "tag": "Elon", "id": "1484015617657257985"}], "meta": {"sent": "2022-01-20T04:11:11.337Z", "summary": {"created": 4, "not_created": 0, "valid": 4, "invalid": 0}}}
200
NEW tesla: https://youtu.be/vtZfQnDHGxU around 4:38 0.0
NEW tesla: > The aggressive pricing from Xpeng comes as China's electric vehicle market continues to heat up with an increasing number o